In [1]:
from collections import Counter, OrderedDict,defaultdict
import numpy as np
import string
import re
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Dropout, Embedding
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences 
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import BatchNormalization
from keras.layers.recurrent import LSTM
from keras import optimizers
import tensorflow as tf
import pandas as pd
# import pickle
# from spacy.vocab import Vocab
# from spacy.language import Language
# import spacy

Using TensorFlow backend.


In [2]:
#making files
from_lang='english'
to_lang='english' 
mapping_path='dataset/Cross_word_mapping/'



In [3]:
def data(from_lang):
    
    punc_table = str.maketrans({key: None for key in string.punctuation})
    sentences = []
    targets = []
    count =0;
    
    with open('dataset/'+str(from_lang)+'_word_def.txt', 'r') as filee:
        
        for i, line in enumerate(filee):
            words = line.strip('\n').split('\t')
            word = words[0]
            definitions = words[1].split(';')
            for definition in definitions:
                definition = re.sub("[\(\[].*?[\)\]]", "", definition).replace('  ', ' ')
                temp_word_list = definition.translate(punc_table).lower().split(' ')
                temp_word_list = list(filter(None, temp_word_list))
                mid_sent=['<start>'] + temp_word_list + ['<end>']
                if mid_sent in sentences:
                    continue
                sentences.append(['<start>'] + temp_word_list + ['<end>'])
                targets.append(word)
    WORD=[]
    for word_sublist in sentences:
        for word in word_sublist:
            WORD.append(word)
    words=WORD
    inf = float('inf')
    frequency_dict = OrderedDict({'<end>': inf, '<start>': inf})
    words_frequency_dict = sorted(Counter(words).most_common(None), key=lambda x:x[1], reverse=True)
    
    defs_frequency_dict = sorted(Counter(targets).most_common(None), key=lambda x:x[1], reverse=True)
    frequency_dict.update(words_frequency_dict)
    frequency_dict.update(defs_frequency_dict)
    frequency_dict.move_to_end('<start>', last=False)

    word2idx = OrderedDict([(item[0], i) for i,item in enumerate(frequency_dict.items())])
    idx2word = dict(zip(word2idx.values(), word2idx.keys()))
    
    return sentences, targets, word2idx, idx2word

In [5]:
sentences, targets, word2idx, idx2word = data(from_lang)


In [6]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
model = load_model('model/'+from_lang+'.h5')
# Check its architecture
# model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
data=pd.read_csv("data_lang.csv")
# data=np.array(data)
data_len=len(data)
path="dataset/Cross_word_mapping/"

english_hindi=defaultdict(list)
hindi_english=defaultdict(list)
english_marathi=defaultdict(list)
marathi_english=defaultdict(list)
english_punjabi=defaultdict(list)
punjabi_english=defaultdict(list)
english_bengali=defaultdict(list)
bengali_english=defaultdict(list)
hindi_marathi=defaultdict(list)
marathi_hindi=defaultdict(list)
hindi_punjabi=defaultdict(list)
punjabi_hindi=defaultdict(list)
hindi_bengali=defaultdict(list)
bengali_hindi=defaultdict(list)
marathi_punjabi=defaultdict(list)
punjabi_marathi=defaultdict(list)
marathi_bengali=defaultdict(list)
bengali_marathi=defaultdict(list)
punjabi_bengali=defaultdict(list)
bengali_punjabi=defaultdict(list)

df=data[['english','hindi','marathi','punjabi','bengali']]

def word_mapping(word_line):
    word_line=str(word_line)
    word_line=word_line[1:len(word_line)-1]
    word_line=word_line.split(',')
    word_return=[]
    for word in word_line:
        word=word.strip()
        word=word[1:len(word)-1]
        word_return.append(word)
    return word_return

df=np.array(df)
#preparation for word mapping mapping
for i in range(0,len(df)):
    english=word_mapping(df[i][0])
    hindi=word_mapping(df[i][1])
    punjabi=word_mapping(df[i][3])
    marathi=word_mapping(df[i][2])
    bengali=word_mapping(df[i][4])
    for word in english:
        english_hindi[word].append(hindi)
        english_punjabi[word].append(punjabi)
        english_marathi[word].append(marathi)
        english_bengali[word].append(bengali)
    for word in hindi:
        hindi_english[word].append(english)
        hindi_punjabi[word].append(punjabi)
        hindi_marathi[word].append(marathi)
        hindi_bengali[word].append(bengali)
    for word in punjabi:
        punjabi_hindi[word].append(hindi)
        punjabi_english[word].append(english)
        punjabi_marathi[word].append(marathi)
        punjabi_bengali[word].append(bengali)
    for word in marathi:
        marathi_hindi[word].append(hindi)
        marathi_punjabi[word].append(punjabi)
        marathi_english[word].append(english)
        marathi_bengali[word].append(bengali)
    for word in bengali:
        bengali_hindi[word].append(hindi)
        bengali_punjabi[word].append(punjabi)
        bengali_marathi[word].append(marathi)
        bengali_english[word].append(english)
    

In [8]:
crosslist1=['english','hindi','marathi','punjabi','bengali']
crosslist2=['english','hindi','marathi','punjabi','bengali']
listofdict=[english_hindi,english_punjabi,english_marathi,english_bengali,hindi_english,hindi_punjabi,hindi_marathi\
            ,hindi_bengali,punjabi_hindi,punjabi_english,punjabi_marathi,punjabi_bengali,marathi_hindi,\
            marathi_punjabi,marathi_english,marathi_bengali,bengali_hindi,bengali_punjabi,bengali_marathi,bengali_english]
listofdictname={'english_hindi':0,'english_punjabi':1,'english_marathi':2,'english_bengali':3,'hindi_english':4,\
                'hindi_punjabi':5,'hindi_marathi':6,'hindi_bengali':7,'punjabi_hindi':8,'punjabi_english':9,\
                'punjabi_marathi':10,'punjabi_bengali':11,'marathi_hindi':12,'marathi_punjabi':13,'marathi_english':14,\
                'marathi_bengali':15,'bengali_hindi':16,'bengali_punjabi':17,'bengali_marathi':18,'bengali_english':19}

In [137]:
train=pd.read_csv("data_lang.csv")
train=train[["english","english_gloss"]]
X=np.array(train['english_gloss'])
X_label=np.array(train['english'])



In [138]:
mp={}
for x in X_label:
    temp=x[1:len(x)-1]
    temp=temp.split(',')
    for word in temp:
        word=word[1:len(word)-1]
        mp[word]=x;


In [139]:
data_rich=pd.read_csv("dataset_richa.csv",header=None)
target=np.array(data_rich[0])
defi=np.array(data_rich[1])
test_data=[]
test_label=[]
for i in range(0,len(target)):
    if(target[i] in mp):
        temp=mp[target[i]]
        temp=temp[1:len(temp)-1]
        temp=temp.split(',')
        df=[]
        for word in temp:
            word=word.strip()
            word=word[1:len(word)-1]
            df.append(word)
        df.append(target[i])
        test_data.append(df)
        test_label.append(defi[i])

In [144]:
test_predict=[]
def find_word(definition):
    words = definition.split(' ')
    idxs = []
    meaning=[]
    for word in words:
        if(word in word2idx):
            idxs.append(word2idx[word])
    idxs = np.array([0] + idxs + [1]).reshape((1,len(idxs) + 2))
    prediction = model.predict(idxs, verbose=0)

    index = np.argmax(prediction)
#     print(index)
    
    meaning=idx2word[index]
    test_predict.append(meaning)

In [145]:

for i in test_label:
    find_word(i)


In [146]:
print(len(test_label))

848


In [147]:
for i in range(0,200):
    print(test_data[i],test_predict[i])

['war', 'warfare', 'war'] transaction
['slow', 'slow'] hasty
['money', 'money'] cash
['immediately', 'instantly', 'straightaway', 'straight_off', 'directly', 'now', 'right_away', 'at_once', 'forthwith', 'in_real_time', 'like_a_shot', 'immediately'] over_and_over
['initially', 'ab_initio', 'at_first', 'at_the_start', 'initially'] over_and_over
['nearby', 'nearby'] hen
['time', 'time'] outcaste
['year', 'year'] tax
['people', 'populate', 'people'] populace
['day', 'twenty-four_hours', 'twenty-four_hour_period', '24-hour_interval', 'solar_day', 'mean_solar_day', 'day'] day
['man', 'adult_male', 'man'] humanity
['thing', 'thing'] called
['work', 'work'] adapted
['child', 'kid', 'youngster', 'minor', 'shaver', 'nipper', 'small_fry', 'tiddler', 'tike', 'tyke', 'fry', 'nestling', 'child'] middle_age
['life', 'life'] July
['world', 'human_race', 'humanity', 'humankind', 'human_beings', 'humans', 'mankind', 'man', 'world'] acre
['back', 'backward', 'backwards', 'rearward', 'rearwards', 'back'] 

In [148]:
count=0;
for i in range(0,848):
    if test_predict[i] in test_data[i]:
        count+=1
print(count/100)

0.18


In [79]:
# from sklearn.metrics import accuracy_score
# print(accuracy_score(test_data,test_predict))

In [30]:
temp=listofdict[listofdictname[from_lang+'_'+to_lang]][meaning]
ans=[]
for line in temp:
    for word in line:
        ans.append(word)
        if(len(ans)>4):
            break
    if(len(ans)>4):
        break
print(ans)
        

['पेनाल्टी', 'दंड', 'सज़ा', 'सजा', 'दण्ड']
